In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../datasets/IMDB Dataset.csv')

In [3]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
df.shape

(50000, 2)

In [5]:
from sklearn.model_selection import train_test_split
df_train,df_test= train_test_split(df,test_size=0.2,random_state=18)

In [6]:
df_train.shape

(40000, 2)

In [7]:
review1 = df_train.iloc[0][0]

In [8]:
# lowering down the review
review1 =review1.lower()

In [9]:
#remove html tags
import re
def cleanhtml(raw_html):
    raw_html = raw_html.lower()
    CLEANR = re.compile('<.*?>')
    cleantext = re.sub(CLEANR, '', raw_html)
    return cleantext


In [10]:
df_train['review'] = df_train['review'].apply(cleanhtml)

In [11]:
df_train['review']

20469    pros: akshay kumar's performance(is it just me...
36519    very rarely does one come across an indie come...
292      when i first saw this film around 6 months ago...
7360     i thought it was an excellent movie. gary cole...
29734    the danes character finally let's buddy have t...
                               ...                        
49073    gentle and genial film seems to have been over...
1726     i watched this movie based on the comments of ...
35653    ...which isn't exactly a ringing endorsement. ...
25720    not as bad as some are making it out to be, th...
28947    imagine the most depressing winter you will ne...
Name: review, Length: 40000, dtype: object

In [12]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [13]:
tokenizer = Tokenizer(num_words=25000)
tokenizer.fit_on_texts(df_train['review'])  # word frequesncy

In [14]:
len(tokenizer.index_word)

113612

In [15]:
tokenized_data = tokenizer.texts_to_sequences(df_train['review'])

In [16]:
tokenizer.index_word[7401]

'pros'

In [17]:
len(tokenized_data[0]),len(tokenized_data[1]),len(tokenized_data[2]),len(tokenized_data[3])

(342, 130, 143, 129)

In [18]:
total = 0
for r in tokenized_data:
    total = total + len(r)
total/len(tokenized_data)

225.0148

In [19]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [20]:
padded_data = pad_sequences(tokenized_data,maxlen=225,padding='post',truncating='pre')

In [21]:
padded_data[1]

array([   51,  1720,   123,    26,   215,   624,    31,  2648,   206,
          11,   885,     3,  5383,  1352,  1638,  3016,     6,     3,
        1314,  1520,     1,   563,   163,    20,    60, 15714,     1,
        2963,     4,   951,    23,   198,   156,    17,   410,     3,
         578,   242,    13,    31,   472,  6053,     1,   226,   303,
          31,   222,   162,    29,    20,    60,     1,  5078,     4,
           1,  2766,   563,    17,  7163,    33,    71,   206,    81,
           1,  8491,   181,     4,     1,   563,    11,    21,   183,
         341,    17,  3347,     5,  8122,    13,     3,   107,    51,
         152,   513,   466,     1,  2256,   327,  2029,  1361,  1017,
           5,   195,     1,    18,     3,   586,   162,   237,    20,
        7107,   182,    50,     8,   272,     5,   232,     3,    48,
          16,     9,   183,   843,     5,    61,    47,    27,   272,
          52,    15,   364,    10,   219,    66,    93,    82,   424,
           2,   100,

In [22]:
len(padded_data[0]),len(padded_data[1]),len(padded_data[2]),len(padded_data[3])

(225, 225, 225, 225)

In [23]:
from tensorflow.keras.layers import LSTM, Embedding, Dense
from tensorflow.keras.models import Sequential

In [24]:
model = Sequential()
model.add(Embedding(input_dim=25000, output_dim=80, input_length=225))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100, return_sequences=False))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', metrics=['accuracy'])


In [25]:
y = df_train['sentiment'].apply(lambda x:0 if x=='negative' else 1)

In [ ]:
model.fit(padded_data, y, batch_size=64, epochs=3)

Epoch 1/3
406/625 [==================>...........] - ETA: 10:50 - loss: 0.6618 - accuracy: 0.6003

In [35]:
model.save('my_imdb_model.h5')

In [36]:
df_test['review'] = df_test['review'].apply(cleanhtml)
tokenized_data_test = tokenizer.texts_to_sequences(df_test['review'])
padded_data_test = pad_sequences(tokenized_data_test,
              maxlen=225, padding='post',truncating='pre')
yp = model.predict(padded_data_test)

313/313 [==============================] - 5s 13ms/step


In [39]:
yp = 

array([[0.70516896],
       [0.08674368],
       [0.00706425],
       ...,
       [0.00781667],
       [0.00626189],
       [0.01116759]], dtype=float32)

In [41]:
yp = np.where(yp>0.5,'positive','negative')

In [42]:
from sklearn.metrics import accuracy_score

In [43]:
accuracy_score(df_test['sentiment'],yp)

0.8887

In [51]:
# testng on real data 
def get_sentiments(reviews):
  reviews = pd.Series(reviews)
  reviews = reviews.apply(cleanhtml)
  tokenized_data_test = tokenizer.texts_to_sequences(reviews)
  padded_data_test = pad_sequences(tokenized_data_test,
                maxlen=225, padding='post',truncating='pre')
  yp = model.predict(padded_data_test)

  return yp

In [52]:
reviews = ['Very good movie. The subject is unusual.... Movie shows reality of factory workers and victim of viral videos of those women',
           'Heart warming ☺️ lots of love for One and only Alakh sir🔥🔥',
           'Amazing, literally I crying many of time when I saw this series....Love you Alakh sir 😇😇',
           '''Very few web series like this: Hatts of to Mr. Kakkar sir for making and bringing this masterpiece for us. Outstanding series I've ever watched related to Mumbai.... ''']
           

In [53]:
reviews

['Very good movie. The subject is unusual.... Movie shows reality of factory workers and victim of viral videos of those women',
 'Heart warming ☺️ lots of love for One and only Alakh sir🔥🔥',
 'Amazing, literally I crying many of time when I saw this series....Love you Alakh sir 😇😇',
 "Very few web series like this: Hatts of to Mr. Kakkar sir for making and bringing this masterpiece for us. Outstanding series I've ever watched related to Mumbai.... "]

In [54]:
import numpy as np
import pandas as pd

get_sentiments(reviews)

1/1 [==============================] - 0s 35ms/step


array([[0.9180415 ],
       [0.9125324 ],
       [0.87863153],
       [0.9729884 ]], dtype=float32)

In [57]:
yp = np.where(yp>0.5,['positive'],['negative'])

TypeError: ignored